In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

d:\Study\Anaconda\envs\BERTopic_Env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 若本地有训练好的模型，则可以直接加载
model_path = './二分类model'
# 加载分词器
tokenizer = BertTokenizer.from_pretrained(model_path)
# 加载模型
model = BertForSequenceClassification.from_pretrained(model_path)

# # 加载预训练的BERT模型和分词器
# model_name = 'aspire/acge_text_embedding'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [3]:
# 加载中文情感分析数据集
dataset = load_dataset('lansinuote/ChnSentiCorp')
dataset['test']

# dataset = load_dataset('csv', data_files='../merged.csv')
# # 划分数据集，按 8:1:1 比例划分为训练集、验证集和测试集
# train_test_split = dataset['train'].train_test_split(test_size=0.2)
# test_valid_split = train_test_split['test'].train_test_split(test_size=0.5)
# # 创建包含训练集、验证集和测试集的数据集字典
# dataset = DatasetDict({
#     'train': train_test_split['train'],
#     'validation': test_valid_split['train'],
#     'test': test_valid_split['test'],
# })
# dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 1200
})

In [4]:
# 数据预处理函数
def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

In [5]:
# 应用预处理函数到数据集
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [6]:
# 定义指标计算函数
def compute_metrics(pred):
    # pred是一个对象，包含预测结果。label_ids是模型的真实标签，表示数据集中每个样本的真实情感类别
    labels = pred.label_ids
    # predictions是模型的预测结果，是一个包含每个样本预测得分的二维数组。argmax(-1)表示在最后一个维度上取最大值的索引，即取出每个样本的预测类别。
    # 简单来说，preds是模型预测的情感类别。
    preds = pred.predictions.argmax(-1)
    # average='binary'：指定计算方式。在二分类任务中，使用'binary'表示计算针对正类（通常是标签为1的类）的指标。
    # 其他选项包括'micro', 'macro', 'weighted', 以及'samples'，这些用于多分类任务。
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [7]:
# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results-二分类',         # 输出目录
    evaluation_strategy='epoch',    # 表示每个epoch结束时进行评估
    per_device_train_batch_size=16, # 训练时每个设备上的批次大小
    per_device_eval_batch_size=64,  # 评估时每个设备上的批次大小
    num_train_epochs=3,             # 训练的轮数
    weight_decay=0.01,              # 指定训练过程中的权重衰减为0.01，以防止过拟合
)

In [8]:
# 创建Trainer对象
trainer = Trainer(
    model=model,                                    # 模型
    args=training_args,                             # 训练参数
    train_dataset=encoded_dataset['train'],         # 训练集
    eval_dataset=encoded_dataset['validation'],     # 验证集
    compute_metrics=compute_metrics,                # 评估指标
)

In [ ]:
# 训练模型，若是使用保存在本地且训练好的模型，则不需要此步骤
trainer.train()

In [9]:
# 评估模型
trainer.evaluate()

100%|██████████| 19/19 [00:07<00:00,  2.48it/s]


{'eval_loss': 0.27383413910865784,
 'eval_accuracy': 0.9366666666666666,
 'eval_precision': 0.9480069324090121,
 'eval_recall': 0.9224283305227656,
 'eval_f1': 0.935042735042735,
 'eval_runtime': 8.2136,
 'eval_samples_per_second': 146.099,
 'eval_steps_per_second': 2.313}

In [ ]:
# 保存模型
model.save_pretrained('./二分类model')
tokenizer.save_pretrained('./二分类model')

In [ ]:
# 推理函数
def predict(text):
    # 确定设备
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 将模型移动到相同的设备
    model.to(device)
    
    # 预处理输入文本
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    
    # 将输入数据移动到相同的设备
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # 关闭梯度计算以节省内存和计算力
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 获取预测结果
    predictions = torch.argmax(outputs.logits, dim=-1)
    
    return "Positive" if predictions.item() == 1 else "Negative"


In [ ]:

# 示例推理
example_text = "ChatGPT，我爱你"
print(predict(example_text))


In [ ]:
import pandas as pd
from tqdm import tqdm

# 确定设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 关闭模型的训练模式
model.eval()

# 读取xlsx文件并去重
df = pd.read_excel('ChatGPT_去重.xlsx')  # 替换为你的xlsx文件路径
def check_length(row):
    return len(row['微博博文']) < 200

filtered_df = df[df.apply(check_length, axis=1)]

# 假设文本内容在"微博博文"列
texts = filtered_df['微博博文'].tolist()

# 定义推理函数
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return "Positive" if predictions.item() == 1 else "Negative"

# 创建一个进度条
tqdm.pandas()

# 对每个文本进行情感预测
df['sentiment'] = df['微博博文'].progress_apply(predict)

# 保存结果回xlsx文件
df.to_excel('predicted_sentiments.xlsx', index=False)

print("情感预测完成，并已保存到predicted_sentiments.xlsx")
